In [2]:
import pickle
import pandas as pd
import numpy as np
import torch

# Helpers
from testing import *
# Baseline model 1
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline

# Check if GPU acceleration is available
if torch.cuda.is_available():
    device_num = torch.cuda.current_device()
else:
    # CPU
    device_num = -1

In [3]:
# Run this if you removed pickle files
# saving_pickles()

train_df, valid_df, test_df = loading_pickles()

In [4]:
# Constants

mask_prob = 0.5
window_size = 100
batch_size = 50

In [5]:
# Masking the given code dataframe and use in every testing
# It is necessary to test models fairly (with the same masked variables on the given probability)
masked_code_df = mask_variable_df(valid_df, mask_prob=mask_prob)
merged_code_df = pd.concat([valid_df, masked_code_df], axis="columns")

Masking: 100%|██████████| 23107/23107 [00:09<00:00, 2441.74it/s]


(Currently) two ways to fine tune the model:
1. Fine-tune the last layer of the model
2. Add one extra layer to the model and fine tune the additional layer

### Baseline model 1

In [4]:
model_b1 = RobertaForMaskedLM.from_pretrained('microsoft/codebert-base-mlm')
tokenizer_b1 = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')
fill_mask_b1 = pipeline('fill-mask', model=model_b1, tokenizer=tokenizer_b1, device=device_num)